In [80]:
import streamlit as st
import tweepy as tw
import pandas as pd
import time

In [17]:
# Get column names
api_colnames_call = api.user_timeline(screen_name = 'TSNBobMcKenzie',count=200)
json_data = [c._json for c in api_colnames_call]

# create dataframe from JSON result
api_colnames = pd.json_normalize(json_data)

# print all column names
for col in api_colnames.columns:
    print(col)

created_at
id
id_str
text
truncated
source
in_reply_to_status_id
in_reply_to_status_id_str
in_reply_to_user_id
in_reply_to_user_id_str
in_reply_to_screen_name
geo
coordinates
place
contributors
is_quote_status
retweet_count
favorite_count
favorited
retweeted
lang
entities.hashtags
entities.symbols
entities.user_mentions
entities.urls
user.id
user.id_str
user.name
user.screen_name
user.location
user.description
user.url
user.entities.url.urls
user.entities.description.urls
user.protected
user.followers_count
user.friends_count
user.listed_count
user.created_at
user.favourites_count
user.utc_offset
user.time_zone
user.geo_enabled
user.verified
user.statuses_count
user.lang
user.contributors_enabled
user.is_translator
user.is_translation_enabled
user.profile_background_color
user.profile_background_image_url
user.profile_background_image_url_https
user.profile_background_tile
user.profile_image_url
user.profile_image_url_https
user.profile_banner_url
user.profile_link_color
user.profile_s

In [75]:
def get_user_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    #Adapted by: https://gist.github.com/yanofsky/5436496?fbclid=IwAR12gb56FOxTNI6R3SfiwpnbPpTvKLoeGR3kP0peQ1nGilcwsF8bR0LSVqE
    
    # Set up Twitter API access
    # Define access keys and tokens
    consumer_key = st.secrets['consumer_key']
    consumer_secret = st.secrets['consumer_secret']
    access_token = st.secrets['access_token']
    access_token_secret = st.secrets['access_token_secret']

    auth = tw.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tw.API(auth, wait_on_rate_limit = True)

    #initialize a list to hold all the tweepy Tweets
    alltweets = []  
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=100)
    
    #save most recent tweets
    alltweets.extend(new_tweets)
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    for i in range(2):
            
            #all subsiquent requests use the max_id param to prevent duplicates
            new_tweets = api.user_timeline(screen_name = screen_name,count=100,max_id=oldest)
            
            #save most recent tweets
            alltweets.extend(new_tweets)
            
            #update the id of the oldest tweet less one
            oldest = alltweets[-1].id - 1
            i +=1
    
    #transform the tweepy tweets into a 2D array that will populate the csv 
    outtweets = [[tweet.user.screen_name, tweet.id_str, tweet.created_at, tweet.text, tweet.retweet_count, tweet.favorite_count] for tweet in alltweets]

    #transform 2D array into pandas dataframe
    df_tweets = pd.DataFrame(data=outtweets, columns=['user_name', 'id_str', 'created_at', 'text', 'rt_count', 'fav_count'])

    return df_tweets

In [35]:
df = get_user_tweets("TSNBobMcKenzie")

In [36]:
df.head()

,user_name,id_str,created_at,text,rt_count,fav_count
0,TSNBobMcKenzie,1412003822394875907,2021-07-05 11:01:59,@barrysacks1 @provincesports @rayferrarotsn no...,0,0
1,TSNBobMcKenzie,1411704305309175815,2021-07-04 15:11:49,Congrats to @MikeMcKenzie11 on completing his ...,12,186
2,TSNBobMcKenzie,1411134094210129920,2021-07-03 01:26:00,@tsnjamesduthie First ballot Hall of Famer.,3,117
3,TSNBobMcKenzie,1411121923979984896,2021-07-03 00:37:38,@MikeMcKenzie11 Everyone is a hockey expert. 😂,0,32
4,TSNBobMcKenzie,1411084623313002500,2021-07-02 22:09:25,Substituted 8 hours of scouts interviews yeste...,25,682


In [29]:
df.shape

(400, 5)

In [84]:
# Get list of Twitter accounts
df_accounts = pd.read_csv('../assets/nhl_app_accounts.csv')
df_accounts.head()

,account_id,name,type,company
0,@TSNBobMcKenzie,Bob McKenzie,insider,TSN
1,@FriedgeHNIC,Elliotte Friedman,insider,HNIC
2,@PierreVLeBrun,Pierre Lebrun,insider,TSN
3,@DarrenDreger,Darren Dreger,insider,TSN
4,@TSNHockey,TSN Hockey,company,TSN


In [42]:
df_accounts['clean_account_ids'] = df_accounts['account_id'].str.replace("@", "")

In [43]:
df_accounts

,account_id,name,type,company,clean_account_ids
0,@TSNBobMcKenzie,Bob McKenzie,insider,TSN,TSNBobMcKenzie
1,@FriedgeHNIC,Elliotte Friedman,insider,HNIC,FriedgeHNIC
2,@PierreVLeBrun,Pierre Lebrun,insider,TSN,PierreVLeBrun
3,@DarrenDreger,Darren Dreger,insider,TSN,DarrenDreger
4,@TSNHockey,TSN Hockey,company,TSN,TSNHockey
5,@rayferrarotsn,Ray Ferraro,insider,TSN,rayferrarotsn
6,@CraigJButton,Craig Button,insinder,TSN,CraigJButton
7,@hockeynight,Hockey Night in Canada,company,HNIC,hockeynight
8,@reporterchris,Chris Johnston,insider,HNIC,reporterchris
9,@JeffMarek,Jeff Marek,insider,Sportsnet,JeffMarek


In [48]:
df_user_tweets = pd.DataFrame(columns=['user_name', 'id_str', 'created_at', 'text', 'rt_count', 'fav_count'])
df_user_tweets

,user_name,id_str,created_at,text,rt_count,fav_count


In [45]:
list_accounts = df_accounts['clean_account_ids'].tolist()
list_accounts

['TSNBobMcKenzie',
 'FriedgeHNIC',
 'PierreVLeBrun',
 'DarrenDreger',
 'TSNHockey',
 'rayferrarotsn',
 'CraigJButton',
 'hockeynight',
 'reporterchris',
 'JeffMarek',
 'SportsnetSpec']

In [85]:
df_user_tweets = pd.DataFrame()

for i in range(len(list_accounts)):
    new_user_tweets = get_user_tweets(list_accounts[i])
    df_user_tweets = df_user_tweets.append(new_user_tweets)

df_user_tweets = df_user_tweets.reset_index(drop=True)

df_user_tweets

,user_name,id_str,created_at,text,rt_count,fav_count
0,TSNBobMcKenzie,1412003822394875907,2021-07-05 11:01:59,@barrysacks1 @provincesports @rayferrarotsn no...,0,0
1,TSNBobMcKenzie,1411704305309175815,2021-07-04 15:11:49,Congrats to @MikeMcKenzie11 on completing his ...,12,186
2,TSNBobMcKenzie,1411134094210129920,2021-07-03 01:26:00,@tsnjamesduthie First ballot Hall of Famer.,3,117
3,TSNBobMcKenzie,1411121923979984896,2021-07-03 00:37:38,@MikeMcKenzie11 Everyone is a hockey expert. 😂,0,32
4,TSNBobMcKenzie,1411084623313002500,2021-07-02 22:09:25,Substituted 8 hours of scouts interviews yeste...,25,682
...,...,...,...,...,...,...
92,SportsnetSpec,1405018385851637760,2021-06-16 04:24:21,RT @Sportsnet: The Montreal Canadiens want to ...,2,0
93,SportsnetSpec,1404857793669042179,2021-06-15 17:46:13,When does Latvia play?,0,6
94,SportsnetSpec,1404856797895094272,2021-06-15 17:42:16,RT @Sportsnet: In a season where the top-heavy...,1,0
95,SportsnetSpec,1404684597988761607,2021-06-15 06:18:00,RT @Sportsnet: In a season where the top-heavy...,15,0


In [82]:
# number of tweets per account
df_user_tweets['user_name'].value_counts()

#TODO: figure out why there are only 97 tweets for SportsnetSpec

rayferrarotsn     300
reporterchris     300
JeffMarek         300
DarrenDreger      300
CraigJButton      300
PierreVLeBrun     300
hockeynight       300
FriedgeHNIC       300
TSNBobMcKenzie    300
TSNHockey         300
SportsnetSpec      97
Name: user_name, dtype: int64

In [88]:
new_df = df_user_tweets.reset_index(drop=True)
new_df

,user_name,id_str,created_at,text,rt_count,fav_count
0,TSNBobMcKenzie,1412003822394875907,2021-07-05 11:01:59,@barrysacks1 @provincesports @rayferrarotsn no...,0,0
1,TSNBobMcKenzie,1411704305309175815,2021-07-04 15:11:49,Congrats to @MikeMcKenzie11 on completing his ...,12,186
2,TSNBobMcKenzie,1411134094210129920,2021-07-03 01:26:00,@tsnjamesduthie First ballot Hall of Famer.,3,117
3,TSNBobMcKenzie,1411121923979984896,2021-07-03 00:37:38,@MikeMcKenzie11 Everyone is a hockey expert. 😂,0,32
4,TSNBobMcKenzie,1411084623313002500,2021-07-02 22:09:25,Substituted 8 hours of scouts interviews yeste...,25,682
...,...,...,...,...,...,...
3092,SportsnetSpec,1405018385851637760,2021-06-16 04:24:21,RT @Sportsnet: The Montreal Canadiens want to ...,2,0
3093,SportsnetSpec,1404857793669042179,2021-06-15 17:46:13,When does Latvia play?,0,6
3094,SportsnetSpec,1404856797895094272,2021-06-15 17:42:16,RT @Sportsnet: In a season where the top-heavy...,1,0
3095,SportsnetSpec,1404684597988761607,2021-06-15 06:18:00,RT @Sportsnet: In a season where the top-heavy...,15,0
